In [ ]:
## Study: Rare Variant Analyses in Ancestrally Diverse Cohorts Reveal Novel ADHD Risk Genes
## Analysis: comparison of rare variant distribution within 15 ADHD risk genes in different ancestries
## Author: Seulgi Jung

In [ ]:
import hail as hl
import os

In [ ]:
## read matrix table of the joint-genotyped data of All of Us version 7
## variants in this matrix table were already splitted and QCed following to the AllofUs QC document below. 
## QC document of AllofUs: https://support.researchallofus.org/hc/en-us/articles/4617899955092-All-of-Us-Beta-Release-Genomic-Quality-Report-
mt = hl.read_matrix_table('gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/exome_v7.1/splitMT/hail.mt')

In [ ]:
## Location of genes (hg38)
## KDMB5: chr1:202,724,495-202,808,487
## CTNND2: chr5:10,971,836-11,904,446
## ANTXR1: chr2:69,013,144-69,249,327
## CHD6: chr20:41,402,083-41,618,384
## SNX17: chr2:27,370,496-27,377,535
## RAI1: chr17:17,681,458-17,811,453
## MMP16: chr8:88,032,011-88,328,025
## AGFG1: chr2:227,472,152-227,561,217
## CHST15: chr10:124,007,668-124,093,598
## ZMYND11: chr10:130,088-254,637
## TNFAIP3: chr6:137,866,349-137,883,314
## EP400: chr12:131,949,942-132,080,460
## ST8SIA2: chr15:92,393,881-92,468,728
## IPO5: chr13:97,953,658-98,024,296
## PIK3R2: chr19:18,153,163-18,170,532

In [ ]:
intervals = [hl.parse_locus_interval(x, reference_genome='GRCh38') for x in [
    'chr1:202724495-202808487', 'chr5:10971836-11904446', 'chr2:69013144-69249327', 
    'chr20:41402083-41618384', 'chr2:27370496-27377535', 'chr17:17681458-17811453', 
    'chr8:88032011-88328025', 'chr2:227472152-227561217', 'chr10:124007668-124093598', 
    'chr10:130088-254637', 'chr6:137866349-137883314', 'chr12:131949942-132080460', 
    'chr15:92393881-92468728', 'chr13:97953658-98024296', 'chr19:18153163-18170532'
]]

In [ ]:
mt_gene = hl.filter_intervals(mt, intervals, keep=True)

In [ ]:
mt_gene.write('ADHD_15genes.mt', overwrite=True)

In [ ]:
mt = hl.read_matrix_table('ADHD_15genes.mt')

In [ ]:
flag = hl.import_table(f'gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/qc/flagged_samples.tsv')

In [ ]:
flag = flag.key_by('s')
mt_pheno = mt.annotate_cols(**flag[mt.s])
sampleQC_fail = hl.len(mt_pheno.qc_metrics_filters) > 0
mt_sampleQC_fail = mt_pheno.filter_cols(sampleQC_fail)

In [ ]:
set_to_remove = hl.literal(mt_sampleQC_fail.s.take(549))
mt_sampleQC = mt_pheno.filter_cols(~set_to_remove.contains(mt_pheno['s']))

In [ ]:
lcr = hl.import_bed('LCR-hs38.bed', reference_genome = 'GRCh38')
d_lcr = mt_sampleQC.filter_rows(hl.is_defined(lcr[mt_sampleQC.locus]), keep = False)

In [ ]:
d_lcr.write('ADHD_15genes_lcr.mt')

In [ ]:
mt = hl.read_matrix_table('ADHD_15genes_lcr.mt')

In [ ]:
d = mt.filter_entries((mt.FT == "PASS") | (hl.is_missing(mt.FT)))

In [ ]:
## Manual QC thresholds

ab = d.AD[1] / hl.sum(d.AD)
pab = hl.binom_test(d.AD[1], hl.sum(d.AD), 0.5, "two-sided")


filter_condition_ab = (((d.GT.is_hom_ref()) & (ab <= 0.1) & (d.GQ >= 25)) | 
                       ((d.GT.is_hom_ref()) & hl.is_missing(d.AD) & (d.GQ >= 25)) |
                       ((d.GT.is_het()) & (ab >= 0.3) & (d.GQ >= 25) & (pab >= 1E-09)) |
                       ((d.GT.is_hom_var()) & (ab >= 0.9) & (d.GQ >= 25)))
d_filter = d.filter_entries(filter_condition_ab)
d_filter.write('ADHD_15genes_lcr_filter.mt')

In [ ]:
mt = hl.read_matrix_table('ADHD_15genes_lcr_filter.mt')

In [ ]:
mt_rows = mt.rows()

In [ ]:
mt_rows.export('ADHD_15genes_lcr_filter_rows.txt')

In [ ]:
import os

name_of_file_in_bucket = 'ADHD_15genes_lcr_filter_rows.txt'
my_bucket = os.getenv('WORKSPACE_BUCKET')
os.system(f"gsutil -m cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

In [ ]:
!cat ADHD_15genes_lcr_filter_rows.txt | awk '{print $1"\t"$2}' > ADHD_15genes_lcr_filter_rows_allele.txt
!gzip ADHD_15genes_lcr_filter_rows_allele.txt

In [ ]:
import os
my_bucket = os.getenv('WORKSPACE_BUCKET')

In [ ]:
os.system(f"gsutil -m cp ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv.txt '{my_bucket}/data/'")
os.system(f"gsutil -m cp ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_misB.txt '{my_bucket}/data/'")
os.system(f"gsutil -m cp ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_misA.txt '{my_bucket}/data/'")
os.system(f"gsutil -m cp ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_mis.txt '{my_bucket}/data/'")
os.system(f"gsutil -m cp ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_SYN.txt '{my_bucket}/data/'")

In [ ]:
!cat ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv.txt | grep "LoF=HC" | grep "LoF_flags=SINGLE_EXON;" > ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv_lof1.txt
!cat ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv.txt | grep "LoF=HC" | sed '/LoF_flags/d' > ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv_lof2.txt
!cat ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv_lof1.txt ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv_lof2.txt > ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv_lof.txt

In [ ]:
!cat ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv_lof.txt | grep "CANONICAL=YES" | grep "BIOTYPE=protein_coding" | awk '{print $1"\t"$2"\t"$4"\t"$5"\t"$6"\t"$9"\t"$12}'> ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv_lof_canonical.txt

In [ ]:
!cat ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_misB.txt | grep "CANONICAL=YES" | grep "BIOTYPE=protein_coding" | awk '{print $1"\t"$2"\t"$4"\t"$5"\t"$6"\t"$9"\t"$12}'> ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_misB_canonical.txt
!cat ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_misA.txt | grep "CANONICAL=YES" | grep "BIOTYPE=protein_coding" | awk '{print $1"\t"$2"\t"$4"\t"$5"\t"$6"\t"$9"\t"$12}'> ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_misA_canonical.txt
!cat ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_mis.txt | grep "CANONICAL=YES" | grep "BIOTYPE=protein_coding" | awk '{print $1"\t"$2"\t"$4"\t"$5"\t"$6"\t"$9"\t"$12}'> ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_mis_canonical.txt
!cat ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_SYN.txt | grep "CANONICAL=YES" | grep "BIOTYPE=protein_coding" | awk '{print $1"\t"$2"\t"$4"\t"$5"\t"$6"\t"$9"\t"$12}'> ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_SYN_canonical.txt

In [ ]:
## read the matrix table
mt = hl.read_matrix_table('ADHD_15genes_lcr_filter.mt')

In [ ]:
hl.export_plink(mt, 'ADHD_15genes_lcr_filter', ind_id = mt.s)

In [ ]:
import os

name_of_file_in_bucket = 'ADHD_15genes_lcr_filter.bed'
my_bucket = os.getenv('WORKSPACE_BUCKET')
os.system(f"gsutil -m cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

In [ ]:
import os

name_of_file_in_bucket = 'ADHD_15genes_lcr_filter.bim'
my_bucket = os.getenv('WORKSPACE_BUCKET')
os.system(f"gsutil -m cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

In [ ]:
import os

name_of_file_in_bucket = 'ADHD_15genes_lcr_filter.fam'
my_bucket = os.getenv('WORKSPACE_BUCKET')
os.system(f"gsutil -m cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

In [ ]:
!head ADHD_15genes_lcr_filter.fam

In [ ]:
import os
name_of_file_in_bucket = 'ancestry_preds.txt'
my_bucket = os.getenv('WORKSPACE_BUCKET')
os.system(f"gsutil -m cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

In [ ]:
!cat ancestry_preds.txt | awk '$2 == "eur" {print $1"\t"$1}' > eur.txt
!cat ancestry_preds.txt | awk '$2 == "afr" {print $1"\t"$1}' > afr.txt
!cat ancestry_preds.txt | awk '$2 == "amr" {print $1"\t"$1}' > amr.txt

In [ ]:
## Change kernel from R to python

import os
name_of_file_in_bucket = 'total_ADHD_7198samples.csv'
my_bucket = os.getenv('WORKSPACE_BUCKET')
os.system(f"gsutil -m cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

In [ ]:
!cat total_ADHD_7198samples.csv | sed 's/\,/\t/g' | awk '{print $1"\t"$1}' | sed '1,1'd > total_ADHD_7198samples.txt

In [ ]:
!cp ADHD_15genes_lcr_filter.fam ADHD_15genes_lcr_filter.fam_original

In [ ]:
!cat ADHD_15genes_lcr_filter.fam_original | awk '{print $2"\t"$2"\t"$3"\t"$4"\t"$5"\t""1"}' > ADHD_15genes_lcr_filter.fam

In [ ]:
!plink --noweb --allow-no-sex --bfile ADHD_15genes_lcr_filter --remove total_ADHD_7198samples.txt --make-bed --out ADHD_15genes_lcr_filter_without_ADHD

In [ ]:
!plink --noweb --allow-no-sex --bfile ADHD_15genes_lcr_filter_without_ADHD --keep eur.txt --make-bed --out ADHD_15genes_lcr_filter_without_ADHD_eur
!plink --noweb --allow-no-sex --bfile ADHD_15genes_lcr_filter_without_ADHD --keep amr.txt --make-bed --out ADHD_15genes_lcr_filter_without_ADHD_amr
!plink --noweb --allow-no-sex --bfile ADHD_15genes_lcr_filter_without_ADHD --keep afr.txt --make-bed --out ADHD_15genes_lcr_filter_without_ADHD_afr

In [ ]:
!plink --noweb --allow-no-sex --bfile ADHD_15genes_lcr_filter_without_ADHD_eur --trend --out ADHD_15genes_lcr_filter_without_ADHD_eur_trend
!plink --noweb --allow-no-sex --bfile ADHD_15genes_lcr_filter_without_ADHD_amr --trend --out ADHD_15genes_lcr_filter_without_ADHD_amr_trend
!plink --noweb --allow-no-sex --bfile ADHD_15genes_lcr_filter_without_ADHD_afr --trend --out ADHD_15genes_lcr_filter_without_ADHD_afr_trend

In [ ]:
!head ADHD_15genes_lcr_filter_without_ADHD_eur_trend.model

In [ ]:
!cat ADHD_15genes_lcr_filter_without_ADHD_eur_trend.model | awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$7}' | sed 's/\//\t/g' | sed 's/\:/\t/g' | awk '{print $2"\t"$3"\t"$6"\t"$7"\t"$8"\t"$9}' | sed '1,1d' > ADHD_15genes_lcr_filter_without_ADHD_eur_trend_count.txt
!cat ADHD_15genes_lcr_filter_without_ADHD_amr_trend.model | awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$7}' | sed 's/\//\t/g' | sed 's/\:/\t/g' | awk '{print $2"\t"$3"\t"$6"\t"$7"\t"$8"\t"$9}' | sed '1,1d' > ADHD_15genes_lcr_filter_without_ADHD_amr_trend_count.txt
!cat ADHD_15genes_lcr_filter_without_ADHD_afr_trend.model | awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$7}' | sed 's/\//\t/g' | sed 's/\:/\t/g' | awk '{print $2"\t"$3"\t"$6"\t"$7"\t"$8"\t"$9}' | sed '1,1d' > ADHD_15genes_lcr_filter_without_ADHD_afr_trend_count.txt

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('ADHD_15genes_lcr_filter_without_ADHD_afr_trend_count.txt', header=None, sep="\t")
df.columns=["chr", "pos", "ref", "alt", "Alt_allofus_AFR", "Ref_allofus_AFR"]

df.fillna(0, inplace=True)
print(df)

In [ ]:
df.to_csv('ADHD_15genes_lcr_filter_without_ADHD_afr_trend_count_input.txt', index=False, sep="\t")

In [ ]:
!paste ADHD_15genes_lcr_filter_without_ADHD_eur_trend_count_input.txt \
ADHD_15genes_lcr_filter_without_ADHD_amr_trend_count_input.txt \
ADHD_15genes_lcr_filter_without_ADHD_afr_trend_count_input.txt \
| awk '$4 == $10' | awk '$4 == $16' | awk '$10 == $16' | awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$11"\t"$12"\t"$17"\t"$18}' \
> ADHD_15genes_lcr_filter_without_ADHD_trend_count_input_all.txt

In [ ]:
## Change kernel from R to python
import os
name_of_file_in_bucket = 'ADHD_15genes_lcr_filter_without_ADHD_trend_count_input_all.txt'
my_bucket = os.getenv('WORKSPACE_BUCKET')
os.system(f"gsutil -m cp '{name_of_file_in_bucket}' '{my_bucket}/data/{name_of_file_in_bucket}'")

In [ ]:
case = hl.import_table('ADHD_15genes_lcr_filter_without_ADHD_trend_count_input_all.txt', types={"chr": hl.tstr, "pos": hl.tint32}, impute=True, delimiter='\t')
case = case.annotate(locus_new=hl.locus(case["chr"], case["pos"], reference_genome='GRCh38'))
case = case.annotate(alleles_new = [case.ref, case.alt])
case = case.key_by(case.locus_new, case.alleles_new)

allofus = hl.import_table('trend_count_input_all.txt', types={"chr": hl.tstr, "pos": hl.tint32}, impute=True, delimiter='\t')
allofus = allofus.annotate(locus_new=hl.locus(allofus["chr"], allofus["pos"], reference_genome='GRCh38'))
allofus = allofus.annotate(alleles_new = [allofus.ref, allofus.alt])
allofus = allofus.key_by(allofus.locus_new, allofus.alleles_new)

gnomad = hl.import_table('vep_final_coding_gnomad_freq_rare_final_input_sed_all_chr_mpc.txt', types={"chr": hl.tstr, "pos": hl.tint32}, impute=True, delimiter='\t')
gnomad = gnomad.annotate(locus=hl.locus(gnomad["chr"], gnomad["pos"], reference_genome='GRCh38'))
gnomad = gnomad.annotate(alleles_gnomad = [gnomad.ref, gnomad.alt])
gnomad = gnomad.key_by(gnomad.locus, gnomad.alleles_gnomad)


In [ ]:
case_allofus = case.annotate(**allofus[case.locus_new, case.alleles_new])
case_allofus_filter = case_allofus.filter(hl.is_defined(allofus[case_allofus.key]))

In [ ]:
case_allofus_filter.show()

In [ ]:
case_allofus_filter.count()

In [ ]:
case_allofus_filter.export('ADHD_15genes_allofus.txt')

In [ ]:
case_allofus_filter = hl.import_table('ADHD_15genes_allofus.txt', types={"chr": hl.tstr, "pos": hl.tint32}, impute=True, delimiter='\t')
case_allofus_filter = case_allofus_filter.annotate(locus=hl.locus(case_allofus_filter["chr"], case_allofus_filter["pos"], reference_genome='GRCh38'))
case_allofus_filter = case_allofus_filter.annotate(alleles = [case_allofus_filter.ref, case_allofus_filter.alt])
case_allofus_filter = case_allofus_filter.key_by(case_allofus_filter.locus, case_allofus_filter.alleles)

In [ ]:
case_allofus_filter.show()

In [ ]:
case_allofus_filter = case_allofus_filter.key_by(case_allofus_filter.locus)

In [ ]:
gnomad = gnomad.key_by(gnomad.locus)

In [ ]:
gnomad.show()

In [ ]:
case_allofus_gnomad = case_allofus_filter.annotate(**gnomad[case_allofus_filter.locus])
case_allofus_gnomad_filter = case_allofus_gnomad.filter(hl.is_defined(gnomad[case_allofus_gnomad.key]))

In [ ]:
case_allofus_gnomad_filter.show()

In [ ]:
case_allofus_gnomad_filter.count()

In [ ]:
case_allofus_gnomad_filter.export('ADHD_15genes_allofus_gnomad.txt')


In [ ]:
## Change kernel from R to python
import os
name_of_file_in_bucket = 'ADHD_15genes_allofus_gnomad.txt'
my_bucket = os.getenv('WORKSPACE_BUCKET')
os.system(f"gsutil -m cp '{my_bucket}/data/{name_of_file_in_bucket}' '{name_of_file_in_bucket}' ")

In [ ]:
!cat ADHD_15genes_allofus_gnomad.txt | wc -l

In [ ]:
!cat ADHD_15genes_allofus_gnomad.txt | awk '$20 == $40' | wc -l

In [ ]:
!cat ADHD_15genes_allofus_gnomad.txt | head -2

In [ ]:
## R

In [ ]:
library(MASS)
library(dplyr)
library(data.table)
library(tidyr)
options(stringsAsFactors = F)

In [ ]:
data <- fread("ADHD_15genes_allofus_gnomad.txt",header=TRUE)
nrow(data)

In [ ]:
data <- data %>%
  unite("variant", chr, pos, ref, alt, sep = "_", remove = FALSE)

In [ ]:
data$maf_allofus_NFE <- data$Alt_allofus_NFE/(data$Alt_allofus_NFE + data$Ref_allofus_NFE)
data$maf_allofus_AMR <- data$Alt_allofus_AMR/(data$Alt_allofus_AMR + data$Ref_allofus_AMR)
data$maf_allofus_AFR <- data$Alt_allofus_AFR/(data$Alt_allofus_AFR + data$Ref_allofus_AFR)

In [ ]:
head(data)

In [ ]:
data_rare <- data[(data$AF_nfe <= 0.0001) | (data$AF_amr <= 0.0001) | (data$AF_afr <= 0.0001) | (data$maf_allofus_NFE <= 0.0001) | (data$maf_allofus_AMR <= 0.0001) | (data$maf_allofus_AFR <= 0.0001),]
nrow(data_rare)

In [ ]:
## SYN
list_syn <- read.table("ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_SYN_canonical.txt", header=F)
nrow(list_syn)

In [ ]:
names(list_syn) <- c("chr", "pos", "ref", "alt", "Uploaded_variation", "gene", "consequence")
list_syn <- list_syn %>%
  unite("variant", chr, pos, ref, alt, sep = "_", remove = FALSE)

In [ ]:
data_syn <- data_rare[which(data_rare$variant %in% list_syn$variant),]
nrow(data_syn)

In [ ]:
data_syn_gene <- data_syn %>%
  group_by(Gene) %>%
  summarise(
    Alt_ADHD_NFE_syn = sum(Alt_ADHD_NFE, na.rm = TRUE),
    Alt_ADHD_AFR_syn = sum(Alt_ADHD_AFR, na.rm = TRUE),
    Alt_ADHD_AMR_syn = sum(Alt_ADHD_AMR, na.rm = TRUE),
    AC_nfe_control_syn = sum(Alt_allofus_NFE, AC_nfe, na.rm = TRUE),
    AC_afr_control_syn = sum(Alt_allofus_AFR, AC_afr, na.rm = TRUE),
    AC_amr_control_syn = sum(Alt_allofus_AMR, AC_amr, na.rm = TRUE),
    Alt_allofus_NFE_syn = sum(Alt_allofus_NFE, na.rm = TRUE),
    Alt_allofus_AFR_syn = sum(Alt_allofus_AFR, na.rm = TRUE),
    Alt_allofus_AMR_syn = sum(Alt_allofus_AMR, na.rm = TRUE),
    AC_nfe_syn = sum(AC_nfe, na.rm = TRUE),
    AC_afr_syn = sum(AC_afr, na.rm = TRUE),
    AC_amr_syn = sum(AC_amr, na.rm = TRUE) 
  )

head(data_syn_gene)

In [ ]:
data_syn_gene <- as.data.table(data_syn_gene)

In [ ]:
## PTV
list_ptv <- read.table("ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_ptv_lof_canonical.txt", header=F)
nrow(list_ptv)

In [ ]:
names(list_ptv) <- c("chr", "pos", "ref", "alt", "Uploaded_variation", "gene", "consequence")
list_ptv <- list_ptv %>%
  unite("variant", chr, pos, ref, alt, sep = "_", remove = FALSE)

In [ ]:
data_ptv <- data_rare[which(data_rare$variant %in% list_ptv$variant),]
nrow(data_ptv)

In [ ]:
data_ptv_gene <- data_ptv %>%
  group_by(Gene) %>%
  summarise(
    Alt_ADHD_NFE_ptv = sum(Alt_ADHD_NFE, na.rm = TRUE),
    Alt_ADHD_AFR_ptv = sum(Alt_ADHD_AFR, na.rm = TRUE),
    Alt_ADHD_AMR_ptv = sum(Alt_ADHD_AMR, na.rm = TRUE),
    AC_nfe_control_ptv = sum(Alt_allofus_NFE, AC_nfe, na.rm = TRUE),
    AC_afr_control_ptv = sum(Alt_allofus_AFR, AC_afr, na.rm = TRUE),
    AC_amr_control_ptv = sum(Alt_allofus_AMR, AC_amr, na.rm = TRUE),
    Alt_allofus_NFE_ptv = sum(Alt_allofus_NFE, na.rm = TRUE),
    Alt_allofus_AFR_ptv = sum(Alt_allofus_AFR, na.rm = TRUE),
    Alt_allofus_AMR_ptv = sum(Alt_allofus_AMR, na.rm = TRUE),
    AC_nfe_ptv = sum(AC_nfe, na.rm = TRUE),
    AC_afr_ptv = sum(AC_afr, na.rm = TRUE),
    AC_amr_ptv = sum(AC_amr, na.rm = TRUE) 
  )

head(data_ptv_gene)

In [ ]:
data_ptv_gene <- as.data.table(data_ptv_gene)

In [ ]:
## misB
list_misb <- read.table("ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_misB_canonical.txt", header=F)
nrow(list_misb)

In [ ]:
names(list_misb) <- c("chr", "pos", "ref", "alt", "Uploaded_variation", "gene", "consequence")
list_misb <- list_misb %>%
  unite("variant", chr, pos, ref, alt, sep = "_", remove = FALSE)

In [ ]:
data_misb <- data_rare[which(data_rare$variant %in% list_misb$variant),]
nrow(data_misb)

In [ ]:
data_misb_gene <- data_misb %>%
  group_by(Gene) %>%
  summarise(
    Alt_ADHD_NFE_misb = sum(Alt_ADHD_NFE, na.rm = TRUE),
    Alt_ADHD_AFR_misb = sum(Alt_ADHD_AFR, na.rm = TRUE),
    Alt_ADHD_AMR_misb = sum(Alt_ADHD_AMR, na.rm = TRUE),
    AC_nfe_control_misb = sum(Alt_allofus_NFE, AC_nfe, na.rm = TRUE),
    AC_afr_control_misb = sum(Alt_allofus_AFR, AC_afr, na.rm = TRUE),
    AC_amr_control_misb = sum(Alt_allofus_AMR, AC_amr, na.rm = TRUE),
    Alt_allofus_NFE_misb = sum(Alt_allofus_NFE, na.rm = TRUE),
    Alt_allofus_AFR_misb = sum(Alt_allofus_AFR, na.rm = TRUE),
    Alt_allofus_AMR_misb = sum(Alt_allofus_AMR, na.rm = TRUE),
    AC_nfe_misb = sum(AC_nfe, na.rm = TRUE),
    AC_afr_misb = sum(AC_afr, na.rm = TRUE),
    AC_amr_misb = sum(AC_amr, na.rm = TRUE) 
  )

head(data_misb_gene)

In [ ]:
data_misb_gene <- as.data.table(data_misb_gene)

In [ ]:
## misA
list_misa <- read.table("ADHD_15genes_lcr_filter_rows_locus_allele_vep_annot_mpc_misA_canonical.txt", header=F)
nrow(list_misa)

In [ ]:
names(list_misa) <- c("chr", "pos", "ref", "alt", "Uploaded_variation", "gene", "consequence")
list_misa <- list_misa %>%
  unite("variant", chr, pos, ref, alt, sep = "_", remove = FALSE)

In [ ]:
data_misa <- data_rare[which(data_rare$variant %in% list_misa$variant),]
nrow(data_misa)

In [ ]:
data_misa_gene <- data_misa %>%
  group_by(Gene) %>%
  summarise(
    Alt_ADHD_NFE_misa = sum(Alt_ADHD_NFE, na.rm = TRUE),
    Alt_ADHD_AFR_misa = sum(Alt_ADHD_AFR, na.rm = TRUE),
    Alt_ADHD_AMR_misa = sum(Alt_ADHD_AMR, na.rm = TRUE),
    AC_nfe_control_misa = sum(Alt_allofus_NFE, AC_nfe, na.rm = TRUE),
    AC_afr_control_misa = sum(Alt_allofus_AFR, AC_afr, na.rm = TRUE),
    AC_amr_control_misa = sum(Alt_allofus_AMR, AC_amr, na.rm = TRUE),
    Alt_allofus_NFE_misa = sum(Alt_allofus_NFE, na.rm = TRUE),
    Alt_allofus_AFR_misa = sum(Alt_allofus_AFR, na.rm = TRUE),
    Alt_allofus_AMR_misa = sum(Alt_allofus_AMR, na.rm = TRUE),
    AC_nfe_misa = sum(AC_nfe, na.rm = TRUE),
    AC_afr_misa = sum(AC_afr, na.rm = TRUE),
    AC_amr_misa = sum(AC_amr, na.rm = TRUE) 
  )

head(data_misa_gene)

In [ ]:
data_misa_gene <- as.data.table(data_misa_gene)

In [ ]:
### ADHD cases
adhd_case_eur_n = 3756
adhd_case_amr_n = 518
adhd_case_afr_n = 464


### All of Us controls without ADHD
allofus_eur_n = 129532
allofus_amr_n = 44329 
allofus_afr_n = 47631 

### gnomad
gnomad_eur_n = 622057
gnomad_amr_n = 30019
gnomad_afr_n = 37545


control_eur_n = allofus_eur_n + gnomad_eur_n
control_amr_n = allofus_amr_n + gnomad_amr_n
control_afr_n = allofus_afr_n + gnomad_afr_n


In [ ]:
## AllofUs + gnomad - SYN
# Create a new column for Fisher's exact p-value
data_syn_gene$fisher_nfe_afr_syn <- apply(data_syn_gene, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["AC_afr_control_syn"]), 
                                control_afr_n-as.numeric(row["AC_afr_control_syn"]),
                                as.numeric(row["AC_nfe_control_syn"]),
                                control_eur_n-as.numeric(row["AC_nfe_control_syn"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})

In [ ]:
write.table(data_syn_gene, "ADHD_15genes_syn_comparison_EUR_AFR.txt", col.names=T, row.names=F, sep="\t", quote=F)

In [ ]:
## AllofUs + gnomad - PTV
# Create a new column for Fisher's exact p-value
data_ptv_gene$fisher_nfe_afr_ptv <- apply(data_ptv_gene, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["AC_afr_control_ptv"]), 
                                control_afr_n-as.numeric(row["AC_afr_control_ptv"]),
                                as.numeric(row["AC_nfe_control_ptv"]),
                                control_eur_n-as.numeric(row["AC_nfe_control_ptv"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})



In [ ]:
write.table(data_ptv_gene, "ADHD_15genes_ptv_comparison_EUR_AFR.txt", col.names=T, row.names=F, sep="\t", quote=F)

In [ ]:
## AllofUs + gnomad - misb
# Create a new column for Fisher's exact p-value
data_misb_gene$fisher_nfe_afr_misb <- apply(data_misb_gene, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["AC_afr_control_misb"]), 
                                control_afr_n-as.numeric(row["AC_afr_control_misb"]),
                                as.numeric(row["AC_nfe_control_misb"]),
                                control_eur_n-as.numeric(row["AC_nfe_control_misb"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})



In [ ]:
write.table(data_misb_gene, "ADHD_15genes_misB_comparison_EUR_AFR.txt", col.names=T, row.names=F, sep="\t", quote=F)

In [ ]:
## AllofUs + gnomad - misa
# Create a new column for Fisher's exact p-value
data_misa_gene$fisher_nfe_afr_misa <- apply(data_misa_gene, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["AC_afr_control_misa"]), 
                                control_afr_n-as.numeric(row["AC_afr_control_misa"]),
                                as.numeric(row["AC_nfe_control_misa"]),
                                control_eur_n-as.numeric(row["AC_nfe_control_misa"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})



In [ ]:
write.table(data_misa_gene, "ADHD_15genes_misA_comparison_EUR_AFR.txt", col.names=T, row.names=F, sep="\t", quote=F)

In [ ]:
## PTV+misB+misA

In [ ]:
data_gene_ptv_syn <- merge(data_syn_gene, data_ptv_gene, by="Gene", all.x=TRUE)
data_gene_ptv_syn_misb <- merge(data_gene_ptv_syn, data_misb_gene, by="Gene", all.x=TRUE)
data_gene_total <- merge(data_gene_ptv_syn_misb, data_misa_gene, by="Gene", all.x=TRUE)
data_gene_total[is.na(data_gene_total)] <- 0
head(data_gene_total)

In [ ]:
data_gene_total_merge <- data_gene_total %>%
  group_by(Gene) %>%
  summarise(
    Alt_ADHD_NFE_merge = sum(Alt_ADHD_NFE_ptv, Alt_ADHD_NFE_misb, Alt_ADHD_NFE_misa, na.rm = TRUE),
    Alt_ADHD_AFR_merge = sum(Alt_ADHD_AFR_ptv, Alt_ADHD_AFR_misb, Alt_ADHD_AFR_misa, na.rm = TRUE),
    Alt_ADHD_AMR_merge = sum(Alt_ADHD_AMR_ptv, Alt_ADHD_AMR_misb, Alt_ADHD_AMR_misa, na.rm = TRUE),
    AC_nfe_control_merge = sum(Alt_allofus_NFE_ptv, Alt_allofus_NFE_misb, Alt_allofus_NFE_misa, AC_nfe_ptv, AC_nfe_misb, AC_nfe_misa, na.rm = TRUE),
    AC_afr_control_merge = sum(Alt_allofus_AFR_ptv, Alt_allofus_AFR_misb, Alt_allofus_AFR_misa, AC_afr_ptv, AC_afr_misb, AC_afr_misa, na.rm = TRUE),
    AC_amr_control_merge = sum(Alt_allofus_AMR_ptv, Alt_allofus_AMR_misb, Alt_allofus_AMR_misa, AC_amr_ptv, AC_amr_misb, AC_amr_misa, na.rm = TRUE),
    Alt_allofus_NFE_merge = sum(Alt_allofus_NFE_ptv, Alt_allofus_NFE_misb, Alt_allofus_NFE_misa, na.rm = TRUE),
    Alt_allofus_AFR_merge = sum(Alt_allofus_AFR_ptv, Alt_allofus_AFR_misb, Alt_allofus_AFR_misa, na.rm = TRUE),
    Alt_allofus_AMR_merge = sum(Alt_allofus_AMR_ptv, Alt_allofus_AMR_misb, Alt_allofus_AMR_misa, na.rm = TRUE),
    AC_nfe_merge = sum(AC_nfe_ptv, AC_nfe_misb, AC_nfe_misa, na.rm = TRUE),
    AC_afr_merge = sum(AC_afr_ptv, AC_afr_misb, AC_afr_misa, na.rm = TRUE),
    AC_amr_merge = sum(AC_amr_ptv, AC_amr_misb, AC_amr_misa, na.rm = TRUE) 
  )

head(data_gene_total_merge)

In [ ]:
merge_final <- merge(data_gene_total, data_gene_total_merge, by="Gene")

In [ ]:
# Create a new column for Fisher's exact p-value
merge_final$fisher_nfe <- apply(merge_final, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["Alt_ADHD_NFE_merge"]), 
                                adhd_case_eur_n - as.numeric(row["Alt_ADHD_NFE_merge"]),
                                as.numeric(row["AC_nfe_control_merge"]),
                                control_eur_n - as.numeric(row["AC_nfe_control_merge"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})


In [ ]:
# Create a new column for Fisher's exact p-value
merge_final$fisher_afr <- apply(merge_final, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["Alt_ADHD_AFR_merge"]), 
                                adhd_case_afr_n - as.numeric(row["Alt_ADHD_AFR_merge"]),
                                as.numeric(row["AC_afr_control_merge"]),
                                control_afr_n - as.numeric(row["AC_afr_control_merge"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})

In [ ]:
# Create a new column for Fisher's exact p-value
merge_final$fisher_amr <- apply(merge_final, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["Alt_ADHD_AMR_merge"]), 
                                adhd_case_amr_n - as.numeric(row["Alt_ADHD_AMR_merge"]),
                                as.numeric(row["AC_amr_control_merge"]),
                                control_amr_n - as.numeric(row["AC_amr_control_merge"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})


In [ ]:
#install.packages("DescTools")
library(DescTools)

In [ ]:
# Create a new column for Fisher's exact p-value
merge_final$fisher_nfe_afr_control <- apply(merge_final, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["AC_nfe_control_merge"]), 
                                control_eur_n - as.numeric(row["AC_nfe_control_merge"]),
                                as.numeric(row["AC_afr_control_merge"]),
                                control_afr_n - as.numeric(row["AC_afr_control_merge"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})

merge_final$fisher_nfe_amr_control <- apply(merge_final, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["AC_nfe_control_merge"]), 
                                control_eur_n - as.numeric(row["AC_nfe_control_merge"]),
                                as.numeric(row["AC_amr_control_merge"]),
                                control_amr_n - as.numeric(row["AC_amr_control_merge"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})


merge_final$fisher_afr_amr_control <- apply(merge_final, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["AC_afr_control_merge"]), 
                                control_afr_n - as.numeric(row["AC_afr_control_merge"]),
                                as.numeric(row["AC_amr_control_merge"]),
                                control_amr_n - as.numeric(row["AC_amr_control_merge"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})

In [ ]:

merge_final$bd_nfe_afr <- apply(merge_final, 1, function(row) {
  tryCatch({
    # Validate and sanitize input
    contingency_values <- c(
      as.numeric(row["Alt_ADHD_NFE_merge"]),
      adhd_case_eur_n - as.numeric(row["Alt_ADHD_NFE_merge"]),
      as.numeric(row["AC_nfe_control_merge"]),
      control_eur_n - as.numeric(row["AC_nfe_control_merge"]),
      as.numeric(row["Alt_ADHD_AFR_merge"]),
      adhd_case_afr_n - as.numeric(row["Alt_ADHD_AFR_merge"]),
      as.numeric(row["AC_afr_control_merge"]),
      control_afr_n - as.numeric(row["AC_afr_control_merge"])
    )
    
    # Replace NAs with 0 for safety
    contingency_values[is.na(contingency_values)] <- 0
    
    # Construct 2x2x2 contingency table
    contingency_table <- array(contingency_values, dim = c(2, 2, 2))
    
    # Check dimensions
    if (any(dim(contingency_table) != c(2, 2, 2))) stop("Incorrect matrix dimensions.")
    
    # Perform Mantel-Haenszel test
    bd_test <- BreslowDayTest(contingency_table)
    return(bd_test$p.value)
  }, error = function(e) {
    # Log errors and return NA for failed rows
    message("Error in cmh_nfe_afr: ", e$message)
    return(NA)
  })
})

merge_final$bd_nfe_amr <- apply(merge_final, 1, function(row) {
  tryCatch({
    contingency_values <- c(
      as.numeric(row["Alt_ADHD_NFE_merge"]),
      adhd_case_eur_n - as.numeric(row["Alt_ADHD_NFE_merge"]),
      as.numeric(row["AC_nfe_control_merge"]),
      control_eur_n - as.numeric(row["AC_nfe_control_merge"]),
      as.numeric(row["Alt_ADHD_AMR_merge"]),
      adhd_case_amr_n - as.numeric(row["Alt_ADHD_AMR_merge"]),
      as.numeric(row["AC_amr_control_merge"]),
      control_amr_n - as.numeric(row["AC_amr_control_merge"])
    )
    contingency_values[is.na(contingency_values)] <- 0
    contingency_table <- array(contingency_values, dim = c(2, 2, 2))
    
    if (any(dim(contingency_table) != c(2, 2, 2))) stop("Incorrect matrix dimensions.")
    
    bd_test <- BreslowDayTest(contingency_table)
    return(bd_test$p.value)
  }, error = function(e) {
    message("Error in cmh_nfe_amr: ", e$message)
    return(NA)
  })
})

merge_final$bd_afr_amr <- apply(merge_final, 1, function(row) {
  tryCatch({
    contingency_values <- c(
      as.numeric(row["Alt_ADHD_AFR_merge"]),
      adhd_case_afr_n - as.numeric(row["Alt_ADHD_AFR_merge"]),
      as.numeric(row["AC_afr_control_merge"]),
      control_afr_n - as.numeric(row["AC_afr_control_merge"]),
      as.numeric(row["Alt_ADHD_AMR_merge"]),
      adhd_case_amr_n - as.numeric(row["Alt_ADHD_AMR_merge"]),
      as.numeric(row["AC_amr_control_merge"]),
      control_amr_n - as.numeric(row["AC_amr_control_merge"])
    )
    contingency_values[is.na(contingency_values)] <- 0
    contingency_table <- array(contingency_values, dim = c(2, 2, 2))
    
    if (any(dim(contingency_table) != c(2, 2, 2))) stop("Incorrect matrix dimensions.")
    
    bd_test <- BreslowDayTest(contingency_table)
    return(bd_test$p.value)
  }, error = function(e) {
    message("Error in cmh_afr_amr: ", e$message)
    return(NA)
  })
})


In [ ]:
# Create a new column for Fisher's exact p-value
merge_final$fisher_nfe_afr_control <- apply(merge_final, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["AC_nfe_control_merge"]), 
                                control_eur_n - as.numeric(row["AC_nfe_control_merge"]),
                                as.numeric(row["AC_afr_control_merge"]),
                                control_afr_n - as.numeric(row["AC_afr_control_merge"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})

merge_final$fisher_nfe_amr_control <- apply(merge_final, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["AC_nfe_control_merge"]), 
                                control_eur_n - as.numeric(row["AC_nfe_control_merge"]),
                                as.numeric(row["AC_amr_control_merge"]),
                                control_amr_n - as.numeric(row["AC_amr_control_merge"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})


merge_final$fisher_afr_amr_control <- apply(merge_final, 1, function(row) {
  # Create a 2x2 contingency table
  contingency_table <- matrix(c(as.numeric(row["AC_afr_control_merge"]), 
                                control_afr_n - as.numeric(row["AC_afr_control_merge"]),
                                as.numeric(row["AC_amr_control_merge"]),
                                control_amr_n - as.numeric(row["AC_amr_control_merge"])),
                              nrow = 2)
  # Calculate Fisher's exact test
  fisher_test <- fisher.test(contingency_table)
  # Return the p-value
  return(fisher_test$p.value)
})

In [ ]:
write.table(merge_final, "ADHD_15genes_ptv+misB+misA_merged_comparison.txt", col.names=T, row.names=F, sep="\t", quote=F)